In [1]:
import os, sys, glob, datetime, warnings
import re, csv 
import pandas as pd
import numpy as np
from pathlib import Path
import pycircstat

warnings.filterwarnings('ignore') # ignore all!

# directory to social_navigation_analysis:
user = os.path.expanduser('~')
sys.path.insert(0, '..')
sys.path.insert(0, str(Path(f'{user}/Dropbox/Projects/social_navigation_analysis/social_navigation_analysis'))) 
import preprocess as snt_preprc
from preprocess import load_data, merge_choice_data
import info    

In [8]:
file_path = '/Volumes/synapse/projects/SocialSpace/Projects/SNT-fmri_CUD/QC/SNT_3T_error/Logs/Tests/02.16.23/SocialSpace_Mock021623_v2.log'
experimenter = 'kb'
out_fname = '02-16-23_button-timing.xlsx'

In [4]:
decision_slides = info.decision_trials['cogent_slide_num'].values
from preprocess import merge_choice_data
import re, csv 
import info

#---------------------------------------------------------------

file_path = Path(file_path)
sub_id    = re.split('_|\.', file_path.name)[1] # expects a file w/ snt_subid

# key presses differed across iterations of the task: 
# - these versions had a fixed choice order across subjects
experimenter = experimenter.lower()
keys = ['29','30']
tr_key = '54'

# Read input data into data variable - a list of all the rows in input file
# Each data row has 4 or 8 items, for example:
    #['432843', '[1]', ':', 'Key', '54', 'DOWN', 'at', '418280  ']
    #['384919', '[3986]', ':', 'slide_28_end: 384919']
with open(file_path, 'r') as csvfile: 
    data = [row for row in csv.reader(csvfile, delimiter = '\t') if len(row) == 8 or len(row) == 4]

In [5]:
# the first time the first character's image is shown
first_img  = [row for row in data if row[3].startswith('pic_1_start')][0] 
task_start = int(first_img[3].split()[1])

# the button presses
onsets, downs, ups = [], [], []
for row in data: 
    if ('start' in row[3]):
        slide = row[3].split(':')[0].split('_start')[0]
        onset = (int(row[3].split(':')[1]) - task_start) / 1000
        onsets.append([slide, onset])
    if (row[3] == 'Key' and row[5] == 'DOWN') and (row[4] in keys):
        downs.append([int(row[4]), (int(row[7]) - task_start) / 1000])
    if (row[3] == 'Key' and row[5] == 'UP') and (row[4] in keys):
        ups.append([int(row[4]), (int(row[7]) - task_start) / 1000])

downs_df  = pd.DataFrame(downs, columns=['button', 'time(s)']).astype(float)
ups_df    = pd.DataFrame(ups, columns=['button', 'time(s)']).astype(float)
onsets_df = pd.DataFrame(onsets, columns=['slide', 'time(s)'])

In [9]:
response_df = pd.DataFrame(columns=['slide', 'decision', 'time(s)', 'time_from_onset', 'button', 'press'])

for press_type, df in {'down': downs_df, 'up': ups_df}.items():
    for d, resp in df.iterrows():
        
        # find trials w/ onset < button press time
        possible = onsets_df['time(s)'].values < resp['time(s)']
        if np.sum(possible) > 0:
            slide = onsets_df['slide'][possible].values[-1]
            if slide in decision_slides: decision = True
            else:                        decision = False
            slide_onset = onsets_df['time(s)'][possible].values[-1] # slide onset
            response_df.loc[len(response_df), :] = [slide, decision, resp['time(s)'], resp['time(s)'] - slide_onset, resp['button'], press_type]

response_df.sort_values(by='time(s)', inplace=True)
response_df.reset_index(drop=True, inplace=True)
print(response_df.to_markdown())

# output file
if not os.path.exists(out_fname):
    response_df.to_excel(out_fname, index=False)

|     | slide     | decision   |   time(s) |   time_from_onset |   button | press   |
|----:|:----------|:-----------|----------:|------------------:|---------:|:--------|
|   0 | pic_3     | False      |    13.039 |             1.059 |       30 | down    |
|   1 | pic_3     | False      |    13.319 |             1.339 |       30 | up      |
|   2 | pic_4     | False      |    18.98  |             1.01  |       30 | down    |
|   3 | pic_4     | False      |    19.225 |             1.255 |       30 | up      |
|   4 | pic_5     | False      |    24.945 |             0.985 |       30 | down    |
|   5 | pic_5     | False      |    25.19  |             1.23  |       30 | up      |
|   6 | pic_6     | False      |    30.835 |             0.885 |       30 | down    |
|   7 | pic_6     | False      |    31.09  |             1.14  |       30 | up      |
|   8 | slide_4   | True       |    56.898 |             0.993 |       30 | down    |
|   9 | slide_4   | True       |    57.123 |          

In [137]:
# compare it to the .mat
from scipy import io
mat = io.loadmat(f'{user}/Dropbox/error/SocialSpace_01-23-23_v2.mat')
mat['responses']

array([[array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[29]], dtype=uint8),
        array([[29]], dtype=uint8), array([[30]], dtype=uint8),
        array([[30]], dtype=uint8), arra